In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import sklearn.preprocessing
from sklearn.model_selection import train_test_split

import keras.layers as layers
import keras.models as models

import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np

import os

In [ ]:
!unzip dataset_otimizado.zip

In [ ]:
CATEGORIAS = ['com_mascara', 'sem_mascara']
dados = []
rotulos = []

In [ ]:
# PEGAR AS IMAGENS CONTIDAS NO DATASET
for categorias in CATEGORIAS:
  pacote = os.path.join('dataset_otimizado/', categorias)
  for img in os.listdir(pacote): # PERCORRER CADA IMAGEM
    img_path = os.path.join(pacote, img)
    image = keras.preprocessing.image.load_img(img_path, target_size=(224, 224)) # CARREGANDO IMAGENS E COLOCANDO TODAS NO MESMO TAMANHO
    image = keras.preprocessing.image.img_to_array(image) # TRANSFORMAR CADA IMAGEM EM UM ARRAY DOS VALORES DO PIXEL
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)

    dados.append(image)
    rotulos.append(categorias)

In [ ]:
rotulo_binary = sklearn.preprocessing.LabelBinarizer()
rotulos = rotulo_binary.fit_transform(rotulos)
rotulos = keras.utils.to_categorical(rotulos)

dados = np.array(dados, dtype='float32')
rotulos = np.array(rotulos)
(treinamento_images, teste_images, treinamento_labels, test_labels) = train_test_split(dados, rotulos, test_size=0.20, stratify=rotulos, random_state=42)

In [ ]:
print(treinamento_images[0].shape)
print(treinamento_labels[0])

print(teste_images[0].shape)
print(test_labels[0])

(224, 224, 3)
[0. 1.]
(224, 224, 3)
[1. 0.]


In [ ]:
modelo_base = keras.applications.MobileNetV2(input_tensor=keras.Input(shape=(224, 224, 3)),
                                             include_top=False, weights='imagenet')

input = modelo_base.output
model = keras.layers.AveragePooling2D(pool_size=(7, 7))(input)
model = keras.layers.Dense(128, activation=tf.nn.sigmoid)(input)
model = keras.layers.Dropout(0.5)(model)
output = keras.layers.Dense(1, activation='softmax')(model)                            

model = keras.Model(inputs=modelo_base.input, outputs=output)

In [ ]:
model.compile(loss=keras.losses.binary_crossentropy, 
              optimizer=keras.optimizers.Adam(), 
              metrics=["accuracy"])

In [ ]:
# TREINANDO O MODELO
model.fit(treinamento_images, treinamento_labels, batch_size=32, epochs=20)

In [ ]:
# FAZENDO PREDIÇÕES
predicoes = model.predict(teste_x)

for i in range(5):
  plt.figure(figsize=(15,15))
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.imshow(teste_x[i])
plt.show()

In [ ]:
(loss, acc) = model.evaluate(teste_x, test_y)
print(acc)

7/7 [==============================] - 8s 1s/step - loss: 5.3623 - accuracy: 0.6634
0.6633663177490234
